## GAN Example to generate CIFAR-10 data
- Ref - https://towardsdatascience.com/using-gans-to-generate-realistic-images-using-keras-and-the-cifar10-dataset-7dc6d23de994

In [2]:
import numpy as np
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, Reshape, LeakyReLU
from tensorflow.keras.optimizers import Adam

### Constant

In [7]:
INPUT_SHAPE = (32, 32, 3)
NOISE_DIM = 100
EPOCHS = 100
BATCH_SIZE = 256

### Load Dataset

In [6]:
# Train -- (50000, 32, 32), (50000, )
# Test -- (10000, 28, 28), (10000, )
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 11s 0us/step


### Data Preprocessing

In [8]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_test = (X_test.astype(np.float32) - 127.5) / 127.5

### Data Generator

In [12]:
def generate_real_img(n_samples):
    x_real = X_train[np.random.randint(0, X_train.shape[0], n_samples)]
    y_real = np.ones((n_samples, 1))

    return x_real, y_real

In [13]:
def generate_img_using_model(generator, n_samples):
    noise = np.random.randn(n_samples, NOISE_DIM)

    x_fake = generator.predict(noise)
    y_fake = np.zeros((n_samples, 1))

    return x_fake, y_fake

### Create Model

In [16]:
# Discriminator
def building_discriminator(input_shape=(32, 32, 3)):
    model_d = Sequential()
    # Layer 1 (32 * 32 * 3 -> 32 * 32 * 64)
    model_d.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model_d.add(LeakyReLU(alpha=0.2))

    # Layer 2 (32 * 32 * 64 -> 16 * 16 * 128)
    model_d.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model_d.add(LeakyReLU(alpha=0.2))

    # Layer 3 (16 * 16 * 128 -> 8 * 8 * 128)
    model_d.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model_d.add(LeakyReLU(alpha=0.2))

    # Layer 4 (8 * 8 * 128 -> 4 * 4 * 256)
    model_d.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model_d.add(LeakyReLU(alpha=0.2))

    # Final Classification Layer (4 * 4 * 256 -> 4096 * 1)
    model_d.add(Flatten())
    model_d.add(Dropout(0.4))
    model_d.add(Dense(1, activation='sigmoid'))

    # Compile
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model_d.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model_d

In [4]:
# Generator
def building_generator(latent_dim):
    model_g = Sequential()
    # Layer 1 (100 * 1 -> 4 * 4 * 256)
    model_g.add(Dense(256 * 4 * 4, input_shape=(latent_dim, )))
    model_g.add(LeakyReLU(alpha=0.2))
    model_g.add(Reshape((4, 4, 256)))

    # Layer 2 (4 * 4 * 256 -> 8 * 8 * 128)
    model_g.add(Conv2DTranspose(128, (4, 4), (2, 2), padding='same'))
    model_g.add(LeakyReLU(alpha=0.2))

    # Layer 3 (8 * 8 * 128 -> 16 * 16 * 128 -> 32 * 32 * 3)
    model_g.add(Conv2DTranspose(128, (4, 4), (2, 2), padding='same'))
    model_g.add(LeakyReLU(alpha=0.2))
    model_g.add(Conv2DTranspose(128, (4, 4), (2, 2), padding='same'))
    model_g.add(LeakyReLU(alpha=0.2))    
    model_g.add(Conv2D(3, (3, 3), activation='tanh', padding='same'))

    return model_g    

In [5]:
def building_gan(generator, discriminator):
    gan = Sequential()
    discriminator.trainable = False

    gan.add(generator)
    gan.add(discriminator)

    # Train the generator to make discriminator output real result
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=opt)

    return gan

In [17]:
# Create GAN Model
gen = building_generator(NOISE_DIM)
dis = building_discriminator(INPUT_SHAPE)
gan = building_gan(gen, dis)
gan.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 32, 32, 3)         1466115   
_________________________________________________________________
sequential_4 (Sequential)    (None, 1)                 522497    
Total params: 1,988,612
Trainable params: 1,466,115
Non-trainable params: 522,497
_________________________________________________________________


### Train Model

In [18]:
def training_gan(gan_model, discriminator, generator, batch_size=256, epochs=100):
    for epoch in range(epochs):
        for step in range(X_train.shape[0] // batch_size):
            # Get real and fake image data
            X_real, y_real = generate_real_img(batch_size)
            X_fake, y_fake = generate_img_using_model(generator, batch_size)

            # Concatenate to get training data
            X_batch = np.concatenate([X_real, X_fake], axis=0)
            y_batch = np.concatenate([y_real, y_fake], axis=0)

            # Train discriminator
            d_loss, d_acc = discriminator.train_on_batch(X_batch, y_batch)

            # Generate noise input for GAN training
            X_gan = np.random.randn(batch_size, NOISE_DIM)
            y_gan = np.ones((batch_size, 1))

            # Train GAN (With fixed discriminator)
            gan_loss = gan_model.train_on_batch(X_gan, y_gan)

            # Print loss info at each epoch end
            print('Training progress in epoch #%d step #%d, discriminator loss=%.3f , generator loss=%.3f' % (epoch, step, d_loss, gan_loss))

In [ ]:
training_gan(gan, dis, gen, BATCH_SIZE, EPOCHS)

### Evaluation

In [20]:
# Evaluate Discriminator
X_real, y_real = generate_real_img(BATCH_SIZE)
_, acc_real = dis.evaluate(X_real, y_real)

8/8 [==============================] - 0s 51ms/step - loss: 0.1871 - accuracy: 0.9453


In [21]:
# Evaluate Generator / Discriminator
X_fake, y_fake = generate_img_using_model(gen, BATCH_SIZE)
_, acc_fake = dis.evaluate(X_fake, y_fake)

8/8 [==============================] - 0s 53ms/step - loss: 0.4049 - accuracy: 1.0000


In [ ]:
# Visualize generator result
X_test = np.random.randn(10, NOISE_DIM)
X_fake = gen.predict(X_test)

fig, axs = plt.subplots(2, 5)
for i in range(2):
    for j in range(5):  
        axs[i,j].imshow(np.clip(X_fake[i+j] * 127.5 + 127.5, 0, 255).astype(np.uint8))